## Cropping from Raw to extract eyes open and eyes closed recording
#### Input: *.edf with recording data and the .csv containing timestamps for resting recording
####  Output: *-resting_EC-raw.fif, *resting_EO-raw.fif, *-noise_EO-raw.fif

In [1]:
import sys
#sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
sys.path.append('/media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/Rachel Wu/eeg-projects/Code/eeg_toolkit/')
import os
import mne
import pandas as pd
#%conda install openpyxl -y
import eeg_toolkit
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/sb10flpc002/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/sb10flpc002/mne_data/MNE-fsaverage-data/fsaverage


In [9]:
# Settings
times_tup,time_win_path = preprocess.get_time_window(5)

# Read in csv files with timestamps for eyes open/closed/stop
csv_path = '../../../Data/Eyes Timestamps/'

# Read in processed raw data
data_path = '../../Data/Processed Data/'
epo_path = '../../Data/Processed Data/' + time_win_path
save_path_resting = '../../Data/Processed Data/'
save_path_cont = '../../Data/STC_Continuous/'
save_path_zepo = '../../Data/STC_Z_scored_Epochs/' + time_win_path

if not os.path.exists(save_path_resting):
    os.makedirs(save_path_resting)
    
if not os.path.exists(save_path_cont):
    os.makedirs(save_path_cont)

if not os.path.exists(save_path_zepo):
    os.makedirs(save_path_zepo)

# Noise cov: crop raw during eyes-open resting condition
noise_cov_win = 5.5, 7.5 # rest_min, rest_max

[-2.5,0.0,2.5]


In [6]:
# viable subjects
sub_ids = utils.import_subs(data_path)
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [7]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 36
Controls: 25
Total: 61


In [ ]:
# Import raw data
data_folder = os.listdir(data_path)


In [ ]:
## Function to create recording of the full resting EEG
# Note: use preprocess.to_raw() before 
# sub_id: subject file name ie. 008 or C20
# csv_path: path to the folder with the resting eeg timestamps
# raw_file: *.fif from the preprocess.to_raw() function
def crop_resting(sub_id, csv_path, raw_file):
    #Import CSV file
    csv_folder = os.listdir(csv_path)
    data_folder = os.listdir(data_path)
    
    # Find the corresponding CSV file with the resting EEG timestamps
    timestamp_csv = None
    for file in csv_folder:
        if file.endswith(".csv") and sub_id in file:
            file_path = os.path.join(csv_path, file)
            timestamp_csv = pd.read_csv(file_path)
            break
    
    if timestamp_csv is not None:
        print(f"CSV file for subject is found.")
    else:
        print(f"CSV file with {sub_id} not found in the folder.")

    # Take in the timestamps into new variables
    EC_start = timestamp_csv['Seconds'][0]
    EC_stop = timestamp_csv['Seconds'][1]
    EO_start = timestamp_csv['Seconds'][2]
    EO_stop = timestamp_csv['Seconds'][3]
    
    ## Create timestamps for EO and noise
    # Establish timestamps assuming recording is long enough for noise = 2 mins, EO = 3 mins
    noise_start = EO_start
    noise_stop = EO_start + 120
    cropped_EO_start = noise_stop
    cropped_EO_stop = noise_stop + 180
    
    ## Make noise capture exactly 1.5 or 2 minutes
    ## Make resting EO exactly 2 or 3 minutes   
    if abs(EO_stop - EO_start) < 300: #Resting recording is less than 5 mins
        if abs(EO_stop - EO_start) < 260: #Resting recording is less than 4.5 mins
            #noise is 2 min, EO is 2 minutes
            noise_stop = EO_start + 120 # Noise is 2 minutes 
            cropped_EO_start = EO_stop - 120 # EO is 2 minutes
            cropped_EO_stop = EO_stop
        else:
            #noise is 1.5 min, EO is 3 minutes
            noise_stop = EO_start + 90 # Noise is 1.5 mins
            cropped_EO_start = noise_stop
            cropped_EO_stop = noise_stop + 180 # EO is 3 minutes
    # Check if eyes closed is greater than 3 mins
    if (EC_stop - EC_start) < 180:
        print(f"Eyes closed is not longer than 3 mins. Length of EC reading is: {EC_stop- EC_start}")
    else:
        EC_stop = EC_start + 180

    # Generate the filename based on the provided convention
    EC_filename = f"{sub_id}_eyes_closed-raw.fif"
    noise_filename = f"{sub_id}_noise-raw.fif"
    EO_filename = f"{sub_id}_eyes_open-raw.fif"
    
    # Create filepath for the new file
    EC_filepath = os.path.join(save_path_resting, EC_filename)
    noise_filepath = os.path.join(save_path_resting, noise_filename)
    EO_filepath = os.path.join(save_path_resting, EO_filename)
    
    # Create cropped files 
    eyes_closed = raw.copy().crop(tmin = EC_start, tmax = EC_stop)
    noise_reading = raw.copy().crop(tmin = noise_start, tmax = noise_stop)
    eyes_open = raw.copy().crop(tmin = cropped_EO_start, tmax = cropped_EO_stop)
    
    # Save the cropped raw data to a raw.fif file
    EC_cropped = eyes_closed.save(EC_filepath, overwrite=True)
    noise_cropped = noise_reading.save(noise_filepath, overwrite=True)
    EO_cropped = eyes_open.save(EO_filepath, overwrite=True)

    return EC_cropped, noise_cropped, EO_cropped

In [11]:
for sub_id in sub_ids:
    _ = source_localization.to_source(sub_id,data_path,epo_path,
                                  save_path_cont,
                                  save_path_zepo,
                                  roi_names,
                                  times_tup,
                                  noise_cov_win,
                                  include_zepochs=True,
                                  average_dipoles=True)       

%%%%%%%%%%%%%%	11	!! ZEPOCHS !! medialorbitofrontal-rh		%%%%%%%%%%%%%%

C27_preprocessed-epo.fif
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 60 (4 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 64 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 52
combining the current components...
Processing epoch : 2 / 52
combining the current components...
Processing epoch : 3 / 52
combining the current components...
Processing epoch : 4 / 52
combining the current components...
Processing epoch : 5 / 52
combining the current components...
Processing epoch : 6 / 52
combining the current components...
Processing epoch : 7 / 52
combining the current components...
Processing epoch : 8